**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 8 - Open Science Platform - Provenance

In [1]:
import json
import datetime
from loguru import logger
from integration_helpers import get_engine_receipts_result
from ipfs_functions import upload_json_to_ipfs

def process_engine_receipts(input_json_file, output_json_file):
    try:
        # Load the input JSON file
        with open(input_json_file, "r") as f:
            account_data = json.load(f)
        
        result_data = {}
        
        # Iterate through each account and its transaction logs
        for account, transactions in account_data.items():
            result_data[account] = []
            
            for entry in transactions:
                tx_hash = entry["hex_hash"]
                timestamp = entry["timestamp"]
                
                # Convert timestamp to UTC using timezone-aware datetime
                utc_time = datetime.datetime.fromtimestamp(timestamp / 1000, datetime.timezone.utc).isoformat()

                
                # Get decoded result from transaction hash
                decoded_string = get_engine_receipts_result(tx_hash)
                
                # Append new data to the result structure
                result_data[account].append({
                    "address": entry["address"],
                    "hex_hash": tx_hash,
                    "timestamp": timestamp,
                    "timestamp_utc": utc_time,
                    "decoded_string": decoded_string
                })
        
        # Write the output JSON file
        with open(output_json_file, "w") as f:
            output_json = json.dump(result_data, f, indent=4)
            output_json_cid = upload_json_to_ipfs(output_json)
        
        logger.info(f"Decoded results saved to {output_json_file}")
    except Exception as e:
        logger.error(f"Error processing engine receipts: {e}")


In [2]:
input_json_file="logs/account_data.json"

output_json_file="logs/decoded_results.json"

process_engine_receipts(input_json_file, output_json_file)

2025-02-10 15:06:58.402 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-10 15:06:58.408 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000577b226163636f756e745f6964223a22796f75746866756c5f616c62617474616e694074657374222c22646f6d61696e5f6964223a2274657374222c2271756f72756d223a312c226a736f6e5f64617461223a227b7d227d000000000000000000
2025-02-10 15:06:58.408 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                                               W{"account_id":"youthful_albattani@test","domain_id":"test","quorum":1,"json_data":"{}"}         
202